In [ ]:
!git clone https://github.com/anonymousNeurIPS2021submission5254/SupplementaryMaterial.git

In [ ]:
#We pick the best 5 networks from 20 trained networks based on validation performance.
import numpy as np
import pandas as pd
import sys
import os
from time import time
import gc
import copy
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
sys.path.append('/kaggle/working/SupplementaryMaterial')
from MLRNN import MLRNNClassifier

#Data prep
train = pd.read_feather('/kaggle/input/november-rank-gauss/train_rg')
test = pd.read_feather('/kaggle/input/november-rank-gauss/test_rg')
ss= pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')
FEATURES = [feat for feat in train.columns if 'f' in feat]
TARGET = 'target'

#Features from optuna
#Getting Parameter list
common_parameters = {"max_runtime" : 900, "width" : 1024}
#from https://www.kaggle.com/ottpocket/optuna-muddling?scriptVersionId=75812644
MLR1_parameters = copy.deepcopy(common_parameters)
MLR1_parameters["depth"] = 2
MLR1_parameters["learning_rate"] = 0.0006343537924564782
MLR1_parameters["max_iter"] = 741

#Submission
rep = 20 #number of NN
selected = 5


predictions = []
valid_performances = []
for i, rs in enumerate(range(rep)):
    start = time()
    print(f'Network: {i+1} of {rep},', sep=' ')
    model = MLRNNClassifier(random_state = rs, **MLR1_parameters)
    predictions.append(model.fit(train[FEATURES].values, train[TARGET].values).predict_proba(test[FEATURES].values)[:,1].astype(np.float16))
    valid_performances.append(np.max(model.record["validation"]))
    model.delete_model_weights()
    del model
    print(f'Val: {valid_performances[-1] :.4f}, time: {time() - start :.2f}')
ss[TARGET] = np.mean([predictions[index] for index in np.argsort(valid_performances)[-selected:]],axis = 0)

ss.to_csv('ss.csv', index=False)